# Quantum You Know

This project is based on the classical Uno game but with Quantum features.
The game is played with cards either classical ones or with Quantum properties.

We will now explain the different quantum functions we used for our games for a better visualisation of their effects you can go the bottom of the page to see their usage.

In [1]:
# cards.py
class Card:
    def __init__(self, color, value):
        self.color = color
        self.value = value
        self.cardId = None

    def __str__(self):
        return f"{self.color} {self.value}"

    def matches(self, other):
        return self.color == other.color or self.value == other.value or other.color == "Quantum" or self.color == "Quantum"

    def play(self, game):
        # Default card: no special effect, just advance turn
        game.discard_pile.append(self)

### Quantum search card
Below you can see the function used for the quantum search card 
This card uses the Grover algorithm to search over the hands of the other players if they have a card selected by the player

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator


def card_to_index(card: Card, Hand: list[Card]) -> int:
    for i, c in enumerate(Hand):
        print(f"Checking card at index {i}: {c}")
        if c.color == card.color and c.value == card.value:
            print(f"Returning index: {i}")
            return i
    print("Returning index: -1")
    return -1

def index_to_binary(index: int, num_qubits: int) -> str:
    binary = format(index, f"0{num_qubits}b")
    print(f"Returning binary representation: {binary}")
    return binary

def grover_oracle(target: str) -> QuantumCircuit:
    n = len(target)
    oracle = QuantumCircuit(n)
    for i, bit in enumerate(target):
        if bit == '0':
            oracle.x(i)
    oracle.h(n-1)
    oracle.mcx(list(range(n-1)), n-1)
    oracle.h(n-1)
    for i, bit in enumerate(target):
        if bit == '0':
            oracle.x(i)
    oracle.name = "Oracle"
    print(f"Returning oracle circuit: {oracle.name}")
    return oracle

def diffusion_operator(n: int) -> QuantumCircuit:
    diff = QuantumCircuit(n)
    diff.h(range(n))
    diff.x(range(n))
    diff.h(n-1)
    diff.mcx(list(range(n-1)), n-1)
    diff.h(n-1)
    diff.x(range(n))
    diff.h(range(n))
    diff.name = "Diffusion"
    print(f"Returning diffusion operator circuit: {diff.name}")
    return diff

def grover_card_search(player_hand: list[Card], target_card: Card,
                       verbose: bool = True, return_circuits: bool = False):
    num_cards = len(player_hand)
    if num_cards == 0:
        print("Returning False: player hand is empty.")
        return (False, None, None) if return_circuits else False

    num_qubits = (num_cards - 1).bit_length()
    card_index = card_to_index(target_card, player_hand)
    if card_index == -1:
        if verbose:
            print(f"The card {target_card} is not in the Hand.")
        print("Returning False: card not found in hand.")
        return (False, None, None) if return_circuits else False

    target_bin = index_to_binary(card_index, num_qubits)
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))
    qc.append(grover_oracle(target_bin), range(num_qubits))
    qc.append(diffusion_operator(num_qubits), range(num_qubits))
    qc.measure(range(num_qubits), range(num_qubits))

    backend = AerSimulator()
    transpiled = transpile(qc, backend=backend)

    if verbose:
        print("\nOriginal circuit:")
        print(qc.draw(output='text'))
        print("\nTranspiled circuit:")
        print(transpiled.draw(output='text'))

    result_job = backend.run(transpiled, shots=1024)
    result = result_job.result()
    counts = result.get_counts()
    corrected_counts = {k[::-1]: v for k, v in counts.items()}
    top_result = max(corrected_counts, key=corrected_counts.get)

    if verbose:
        print(f"\nGrover result (corrected): {corrected_counts}")
        print(f"Top result (binary): {top_result} -> index {int(top_result, 2)}")

    found = int(top_result, 2) == card_index
    print(f"Returning result: {found}")

    if return_circuits:
        return found, qc, transpiled
    return found


### Quantum balance card
Below you can find the code used to define the quantum balance card
This card takes the hands of two players and uses QAOA to balance the values of the two hands before before distributing them back to the players

In [3]:
from qiskit_aer import AerSimulator
from qiskit_optimization.translators import from_docplex_mp, to_ising
from docplex.mp.model import Model
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import QAOAAnsatz
from qiskit_ibm_runtime import EstimatorV2 as Estimator, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from scipy.optimize import minimize
import numpy as np



class quantum_balance_card(Card):
    def __init__(self, color="Purple"):
        super().__init__(color, "Quantum Balance")
        self.cardId = 10

    def card_weight(self, card):
        if isinstance(card.value, int):
            weight = card.value
        elif card.value in ['Skip', 'Reverse']:
            weight = 10
        elif card.value == 'Draw Two':
            weight = 15
        elif card.value == 'Quantum':
            weight = 20
        elif card.value == 'DUMMY':
            weight = 0
        else:
            weight = 5
        print(f"Card weight for {card}: {weight}")
        return weight

    def play(self, game, verbose: bool = True, return_circuits: bool = False):
        players = [game.get_current_player(), game.get_next_player()]
        all_cards = players[0].Hand + players[1].Hand

        dummy_used = False
        dummy_index = -1
        if len(all_cards) % 2 != 0:
            dummy_card = Card("Dummy", "DUMMY")
            all_cards.append(dummy_card)
            dummy_used = True
            dummy_index = len(all_cards) - 1

        n_cards = len(all_cards)
        mdl = Model(name="Card_Player_Assignment")
        x = [mdl.binary_var(name=f"x_{i}") for i in range(n_cards)]

        weights = [self.card_weight(card) for card in all_cards]
        avg_weight = sum(weights) / 2
        weight_diff = mdl.sum(weights[i] * x[i] for i in range(n_cards)) - avg_weight
        obj_weight = weight_diff * weight_diff

        card_diff = mdl.sum(x) - n_cards / 2
        obj_card = card_diff * card_diff

        alpha = 40
        mdl.minimize(obj_weight + alpha * obj_card)

        qp = from_docplex_mp(mdl)
        qubo = QuadraticProgramToQubo().convert(qp)
        hamiltonian, _ = to_ising(qubo)

        backend = AerSimulator()
        estimator = Estimator(mode=backend)
        pm = generate_preset_pass_manager(backend=backend, optimization_level=3)

        def cost_function(params, estimator, circuit, hamiltonian):
            isa_psi = pm.run(circuit)
            isa_observables = hamiltonian.apply_layout(isa_psi.layout)
            job = estimator.run([(isa_psi, isa_observables, params)])
            cost = job.result()[0].data.evs
            if verbose:
                print(f"Cost function value: {cost}")
            return cost

        circuit_qaoa = QAOAAnsatz(hamiltonian, reps=20)
        p = circuit_qaoa.num_parameters // 2
        gamma_init = np.linspace(0.1, 1.5, p)
        beta_init = np.linspace(0.1, 1.5, p)
        params_init = np.concatenate([gamma_init, beta_init])

        if verbose:
            print("\nInitial QAOA ansatz circuit:")
            print(circuit_qaoa.draw(output='text'))

        res_opt = minimize(
            cost_function,
            params_init,
            args=(estimator, circuit_qaoa, hamiltonian),
            method="COBYLA"
        )

        params_opt = res_opt.x
        if verbose:
            print(f"Optimized parameters: {params_opt}")

        sampler = Sampler(mode=backend)
        circuit_qaoa_copy = circuit_qaoa.decompose(reps=2).copy()
        circuit_qaoa_copy.measure_all()

        if verbose:
            print("\nMeasurement circuit:")
            print(circuit_qaoa_copy.draw(output='text'))

        counts = sampler.run([(circuit_qaoa_copy, params_opt)]).result()[0].data.meas.get_counts()
        if verbose:
            print(f"Sampler counts: {counts}")

        if isinstance(counts, dict):
            most_likely_bitstring = max(counts, key=counts.get)
        else:
            most_likely_bitstring = next(iter(counts), None)

        if most_likely_bitstring is None:
            raise RuntimeError("Failed to get a bitstring from sampler result.")

        if verbose:
            print(f"Most likely bitstring: {most_likely_bitstring}")

        new_hands = {0: [], 1: []}
        for i in range(n_cards):
            if dummy_used and i == dummy_index:
                continue
            bit_val = int(most_likely_bitstring[::-1][i])
            new_hands[bit_val].append(all_cards[i])

        for j, player in enumerate(players):
            player.Hand = new_hands[j]
        if verbose:
            print(f"New hands assigned: {new_hands}")

        if return_circuits:
            return new_hands, circuit_qaoa, circuit_qaoa_copy
        return new_hands


### The Quantum card
The quantum card uses a one qubit circuit to which we apply an hadamard gate and then measure it
The outcome determines the effect we apply either a pass or reverse effect

In [4]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

class quantum_card(Card):
    def __init__(self, color, value):
        super().__init__(color, value)
        # Pre-generate and store the basic circuit
        self.circuit = self.create_circuit()
        self.cardId = 11  # Unique identifier for this card type

    def create_circuit(self):
        qc = QuantumCircuit(1)
        qc.h(0)  # Put qubit in superposition
        qc.measure_all()
        print(f"Created quantum circuit: {qc.name if qc.name else qc}")
        return qc

    def run_quantum_effect(self, verbose: bool = True, return_circuits: bool = False):
        backend = AerSimulator()
        # (Re)create and transpile the circuit each run
        qc = self.create_circuit()
        transpiled = transpile(qc, backend=backend)

        if verbose:
            print("\nOriginal effect circuit:")
            print(qc.draw(output='text'))
            print("\nTranspiled effect circuit:")
            print(transpiled.draw(output='text'))

        # Execute single-shot measurement
        result = backend.run(transpiled, shots=1).result()
        counts = result.get_counts()
        if verbose:
            print(f"Quantum effect result counts: {counts}")

        if return_circuits:
            return counts, qc, transpiled
        return counts

    def play(self, game, verbose: bool = True, return_circuits: bool = False):
        # Execute base play behavior
        super().play(game)

        # Run the quantum effect, optionally retrieving circuits
        if return_circuits:
            counts, effect_circuit, transpiled = self.run_quantum_effect(verbose, return_circuits)
        else:
            counts = self.run_quantum_effect(verbose)

        if verbose:
            print(f"Counts from quantum effect: {counts}")

        # Apply the card effect based on measurement
        if '0' in counts:
            game.skip_next_player = True
            if verbose:
                print("Effect: Skipping next player.")
        else:
            game.reverse_turn_order()
            if verbose:
                print("Effect: Reversing turn order.")

        if return_circuits:
            return counts, effect_circuit, transpiled
        return counts


### Quantum color card
The quantum color card represents the differents colors as a grpah and applies a quantum walker algorithm to determine which color will have to be played afterwards

In [5]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

class Quantum_color_card(Card):
    """
    Uses a full 1D discrete-time coined quantum walk to choose a color.
    """

    def __init__(self, color_list, current_color="Quantum"):
        self.colors = color_list
        self.current_color = current_color
        self.cardId = 12
        super().__init__(current_color, "Color")

    def play(self, game, verbose: bool = True, return_circuits: bool = False):
        self.current_color = game.get_top_card().color
        if verbose:
            print(f"Current color set to: {self.current_color}")

        if return_circuits:
            new_color, walk_circuit, transpiled = self.activate_quantum_walk(verbose, return_circuits)
        else:
            new_color = self.activate_quantum_walk(verbose)

        if verbose:
            print(f"New color chosen by quantum walk: {new_color}")

        game.current_color = new_color
        self.color = new_color

        if return_circuits:
            return new_color, walk_circuit, transpiled
        return new_color

    def activate_quantum_walk(self, verbose: bool = True, return_circuits: bool = False, steps: int = None):
        """
        Perform a 1D discrete-time quantum walk on 4 colors.
        Position: 2 qubits (q0 = LSB, q1 = MSB), Coin: 1 qubit (q2)
        """
        from qiskit import QuantumCircuit, transpile
        from qiskit_aer import AerSimulator

        color_map = {0: "Red", 1: "Blue", 2: "Green", 3: "Yellow"}
        if steps is None:
            steps = np.random.randint(1, 5)
        if verbose:
            print(f"Quantum walk steps: {steps}")

        qc = QuantumCircuit(3, 2)
        pos_index = self.colors.index(self.current_color)
        if verbose:
            print(f"Initial position index: {pos_index}")
        if pos_index & 0b01:
            qc.x(0)
        if pos_index & 0b10:
            qc.x(1)

        qc.h(2)
        for _ in range(steps):
            qc.h(2)
            qc.x(2)
            self._increment_mod_4(qc, control=2, q0=0, q1=1)
            qc.x(2)
            self._decrement_mod_4(qc, control=2, q0=0, q1=1)

        qc.measure(0, 0)
        qc.measure(1, 1)

        backend = AerSimulator()
        transpiled = transpile(qc, backend)

        if verbose:
            print("\nQuantum walk circuit:")
            print(qc.draw(output='text'))
            print("\nTranspiled walk circuit:")
            print(transpiled.draw(output='text'))

        result = backend.run(transpiled, shots=1).result()
        counts = result.get_counts()
        if verbose:
            print(f"Quantum walk result counts: {counts}")

        measured = list(counts.keys())[0]
        pos = int(measured[::-1], 2)
        color = color_map[pos]
        if verbose:
            print(f"Measured position: {pos}, corresponding to color: {color}")

        if return_circuits:
            return color, qc, transpiled
        return color

    def _increment_mod_4(self, qc, control, q0, q1):
        qc.cx(control, q0)
        qc.ccx(control, q0, q1)

    def _decrement_mod_4(self, qc, control, q0, q1):
        qc.ccx(control, q0, q1)
        qc.cx(control, q0)


### Quantum draw card
The quantum draw card is a quantum circuit that generates a random number between 0 and 7
The number generated determines how many cards the next person has to draw

In [6]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

class Quantum_draw_card(Card):
    def __init__(self, color, max_cards=8):
        super().__init__(color, f"Quantum Draw up to {max_cards}")
        self.max_cards = max_cards
        self.cardId = 13  # Unique identifier for this card type

    def play(self, game, verbose: bool = True, return_circuits: bool = False):
        """Play the quantum draw card effect."""
        # Run the quantum effect to determine number of cards
        if return_circuits:
            drawn_cards, qc, transpiled = self.activate_quantum_effect(verbose, return_circuits)
        else:
            drawn_cards = self.activate_quantum_effect(verbose)

        if verbose:
            print(f"Number of cards to draw: {drawn_cards}")
        
        for _ in range(drawn_cards):
            card = game.deck.DrawCard()
            if card:
                game.get_next_player().Hand.append(card)
                if verbose:
                    print(f"Card drawn and added to next player's hand: {card}")

        if return_circuits:
            return drawn_cards, qc, transpiled
        return drawn_cards

    def activate_quantum_effect(self, verbose: bool = True, return_circuits: bool = False):
        """Generate a quantum random number between 0 and max_cards (inclusive)."""
        n_qubits = 3  # Number of qubits to represent the range
        qc = QuantumCircuit(n_qubits, n_qubits)

        # Put all qubits into superposition
        for q in range(n_qubits):
            qc.h(q)

        # Measure all qubits
        qc.measure(range(n_qubits), range(n_qubits))

        # Use AerSimulator
        sim = AerSimulator()
        transpiled = transpile(qc, sim)

        if verbose:
            print("\nOriginal random-number circuit:")
            print(qc.draw(output='text'))
            print("\nTranspiled random-number circuit:")
            print(transpiled.draw(output='text'))

        # Execute single-shot measurement
        result = sim.run(transpiled, shots=1).result()
        counts = result.get_counts()
        if verbose:
            print(f"Quantum effect result counts: {counts}")

        # Get measured number (Qiskit returns bitstrings MSB->LSB)
        bitstring = list(counts.keys())[0]
        number = int(bitstring[::-1], 2)
        if verbose:
            print(f"Measured bitstring: {bitstring} -> interpreted number: {number}")

        # Rejection sampling if number > max_cards
        if number > self.max_cards:
            if verbose:
                print(f"Number {number} exceeds max_cards {self.max_cards}, retrying...\n")
            # Retry (returns circuits only on the final call)
            return self.activate_quantum_effect(verbose, return_circuits)
        else:
            if verbose:
                print(f"Returning valid number: {number}")
            if return_circuits:
                return number, qc, transpiled
            return number


### Quantum search card
The quantum search cards uses the Grover's algorithm with the functions determined before to search a possessed card through the other player's hands
If the card is found the players concerned wil draw a number of card according to the position of the card found

In [7]:
import random
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

class Quantum_grover_card(Card):
    def __init__(self, color="Purple"):
        super().__init__(color, "Quantum Grover")
        self.cardId = 17  # Just an arbitrary ID
        self.selected_card = None 

    def play(self, game, selected_card=None, show_circuit=True):
        current_player = game.get_current_player()
        other_players = [p for p in game.players if p != current_player]

        if self.selected_card is None:
            # Initial call: UI should trigger selection
            choice = random.randrange(len(current_player.GetHand()))
            self.selected_card = current_player.GetHand()[choice]

        print(f"\n🔍 Grover scan for: {self.selected_card}\n")

        for player in other_players:
            Hand = player.GetHand()
            # Perform Grover search and get both result and circuit
            found, circuit = grover_card_search(Hand, self.selected_card, verbose=False)

            if show_circuit:
                # Display the circuit textually
                print("Quantum circuit used for Grover search:")
                print(circuit.draw(output='text'))

            print(f"Grover search result for {player.GetName()}: {found}")
            if found:
                index = card_to_index(self.selected_card, Hand)
                print(f"✅ {player.GetName()} has {self.selected_card} at index {index}. Adding {index} card(s) as penalty.")
                for _ in range(index):
                    new_card = game.draw_card(game.players.index(player))
                    if new_card:
                        player.AddCard(new_card)
                        print(f"Card added to {player.GetName()}: {new_card}")
            else:
                print(f"❌ {player.GetName()} does not have {self.selected_card}.")

        game.discard_pile.append(self)
        print(f"Card {self} added to discard pile.")
        game.next_turn()
        print("Next turn triggered.")


def grover_card_search(hand, target_card, verbose=False):
    # Map cards to indices
    n = len(hand)
    oracle_index = hand.index(target_card) if target_card in hand else None

    # Initialize quantum circuit
    qc = QuantumCircuit(n, n)

    # Step 1: Apply Hadamard to all qubits
    for q in range(n):
        qc.h(q)

    # Number of Grover iterations
    iterations = int((3.14/4) * (2**(n/2)))

    # Grover iterations
    for _ in range(iterations):
        # Oracle: phase flip the target state
        if oracle_index is not None:
            qc.z(oracle_index)
        # Diffusion operator
        qc.h(range(n))
        qc.x(range(n))
        qc.h(n-1)
        qc.mcx(list(range(n-1)), n-1)  # multi-controlled NOT
        qc.h(n-1)
        qc.x(range(n))
        qc.h(range(n))

    # Measurement
    qc.measure(range(n), range(n))

    if verbose:
        print(qc.draw(output='text'))

    # Execute on AerSimulator
    simulator = AerSimulator()
    transpiled = transpile(qc, simulator)
    result = simulator.run(transpiled, shots=1024).result()
    counts = result.get_counts()

    # Determine most likely index
    most_likely = max(counts, key=counts.get)
    measured_index = int(most_likely, 2)

    found = (measured_index == oracle_index)
    return found, qc


### Quantum shuffle card
The quantum shuffle card collects every players hands 
It shuffles all the cards together and then redistribute a number of cards to every player almost equal to the mean of each players previous number of cards

In [8]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import math

class Quantum_shuffle_card(Card):
    """A card that shuffles all cards from all players using quantum-generated indices."""
    def __init__(self, color, show_circuit=True):
        super().__init__(color, "Quantum True Shuffle")
        self.cardId = 14
        self.show_circuit = show_circuit

    def play(self, game):
        self.quantum_shuffle(game)

    def quantum_shuffle(self, game):
        # Collect all cards from players
        full_deck = []
        for player in game.players:
            full_deck.extend(player.Hand)
            player.Hand.clear()
        print(f"Collected full deck: {[str(card) for card in full_deck]}")

        num_players = len(game.players)
        player_index = 0

        while full_deck:
            # Determine number of qubits required to index current deck
            num_qubits = math.ceil(math.log2(len(full_deck)))
            if num_qubits == 0:
                idx = 0
            else:
                idx, qc = self.quantum_random_index(num_qubits)
                if self.show_circuit:
                    print("Quantum circuit for random index generation:")
                    print(qc.draw(output='text'))
                idx %= len(full_deck)

            print(f"Selected index: {idx} from full deck of size {len(full_deck)}")

            # Assign card to current player
            selected_card = full_deck.pop(idx)
            game.players[player_index].Hand.append(selected_card)
            print(f"Assigned card {selected_card} to player {player_index}")

            player_index = (player_index + 1) % num_players

        print("Shuffling complete. Player hands updated.")

    def quantum_random_index(self, n_qubits):
        """Generates a quantum random integer using n_qubits."""
        qc = QuantumCircuit(n_qubits, n_qubits)
        # Create superposition
        qc.h(range(n_qubits))
        # Measure
        qc.measure(range(n_qubits), range(n_qubits))

        sim = AerSimulator()
        transpiled = transpile(qc, sim)
        result = sim.run(transpiled, shots=1).result()
        counts = result.get_counts()
        print(f"Quantum random index counts: {counts}")

        bitstring = next(iter(counts))
        number = int(bitstring, 2)
        print(f"Returning quantum random index: {number}")
        return number, qc


### Quantum swap card
The quantum swap card uses control swap gates exchange the hands of two players using a superposed control qubit

In [9]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

class Quantum_swap_card(Card):
    """A card that swaps cards of two players using a quantum CSWAP gate, measuring the control and data qubits."""
    def __init__(self, color, max_cards=8, show_circuit=True):
        super().__init__(color, "Quantum Hand Swap")
        self.cardId = 16
        self.max_cards = max_cards
        self.show_circuit = show_circuit

    def play(self, game):
        self.activate_quantum_effect(game, game.get_current_player(), game.get_next_player())

    def activate_quantum_effect(self, game, player1, player2):
        n1, n2 = len(player1.Hand), len(player2.Hand)
        n_swap = min(n1, n2, self.max_cards)

        if n_swap == 0:
            print("[Quantum SWAP] Nothing to swap. Returning.")
            return  # Nothing to swap

        # total qubits = 1 (control) + 2 * n_swap (cards)
        total_qubits = 1 + 2 * n_swap
        qc = QuantumCircuit(total_qubits, total_qubits)

        # Control in superposition
        qc.h(0)

        # Apply CSWAP gates controlled by qubit 0
        for i in range(n_swap):
            qc.cswap(0, 1 + i, 1 + n_swap + i)

        # Measure all qubits
        qc.measure(range(total_qubits), range(total_qubits))

        # Display the circuit if requested
        if self.show_circuit:
            print("\nQuantum CSWAP circuit for hand swap:")
            print(qc.draw(output='text'))

        # Run on simulator
        backend = AerSimulator()
        transpiled_qc = transpile(qc, backend)
        result = backend.run(transpiled_qc, shots=1).result()
        counts = result.get_counts()
        bitstring = next(iter(counts))  # e.g., '0101...'

        print(f"[Quantum SWAP] Measured bitstring: {bitstring}")

        control_bit = bitstring[-1]  # measurement bit 0 is last in returned string

        # Prepare new hands
        new_hand1, new_hand2 = [], []
        for i in range(n_swap):
            if control_bit == '1':
                new_hand1.append(player2.Hand[i])
                new_hand2.append(player1.Hand[i])
            else:
                new_hand1.append(player1.Hand[i])
                new_hand2.append(player2.Hand[i])

        # Append remaining cards
        new_hand1.extend(player1.Hand[n_swap:])
        new_hand2.extend(player2.Hand[n_swap:])

        # Update hands
        player1.Hand = new_hand1
        player2.Hand = new_hand2

        print(f"[Quantum SWAP] {player1.GetName()} new hand: {[str(c) for c in player1.Hand]}")
        print(f"[Quantum SWAP] {player2.GetName()} new hand: {[str(c) for c in player2.Hand]}")


### Quantum teleportation card
The quantum teleportation card uses a bell measurement to determine how many cards should be teleported between two players

In [10]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator  # or from qiskit.providers.aer import AerSimulator
import random

class teleportation_card(Card):
    """A card that teleports a qubit state and uses the measurement to drive swaps between players."""
    def __init__(self, color, show_circuit=True):
        super().__init__(color, "Teleportation")
        self.cardId = 18  # Unique identifier for this card type
        self.show_circuit = show_circuit

    def play(self, game):
        # Run the teleportation circuit
        measurement, qc = self.activate_teleportation()

        if self.show_circuit:
            print("\nQuantum teleportation circuit:")
            print(qc.draw(output='text'))

        print(f"[Teleportation Card] Measurement result: {measurement}")

        # Interpret measurement outcome (bitstring like '00', '01', '10', or '11')
        ones_count = measurement.count('1')
        print(f"[Teleportation Card] Number of '1's in measurement: {ones_count}")

        # For example, swap cards between current and next player equal to number of 1s (max 2)
        num_swaps = ones_count

        if num_swaps == 0:
            print("[Teleportation Card] No swaps to perform. Returning.")
            return

        curr_player = game.get_current_player()
        next_player = game.get_next_player()

        swaps_done = []

        for _ in range(num_swaps):
            if not curr_player.Hand or not next_player.Hand:
                print("[Teleportation Card] One of the players has no cards left to swap. Breaking.")
                break

            # Pick random cards to swap
            card_from_curr = random.choice(curr_player.Hand)
            card_from_next = random.choice(next_player.Hand)

            curr_player.Hand.remove(card_from_curr)
            next_player.Hand.remove(card_from_next)

            curr_player.Hand.append(card_from_next)
            next_player.Hand.append(card_from_curr)

            swaps_done.append((str(card_from_curr), str(card_from_next)))

        msg = f"Teleportation swapped {len(swaps_done)} cards between {curr_player.Name} and {next_player.Name}:\n"
        msg += "\n".join([f"{curr} <--> {nxt}" for curr, nxt in swaps_done])
        print(msg)

    def activate_teleportation(self):
        qc = QuantumCircuit(3, 2)

        # Prepare |+> state to teleport (can be changed)
        qc.h(0)

        # Create Bell pair between qubit 1 and 2
        qc.h(1)
        qc.cx(1, 2)

        # Bell measurement on qubit 0 and 1
        qc.cx(0, 1)
        qc.h(0)

        qc.measure([0, 1], [0, 1])

        # Optionally show the circuit in the activation step
        # Execution
        sim = AerSimulator()
        transpiled_qc = transpile(qc, sim)
        job = sim.run(transpiled_qc, shots=1)
        result = job.result()
        counts = result.get_counts()

        # Return the measurement bitstring (e.g. '00', '01', '10', or '11')
        measurement = next(iter(counts))
        print(f"[Teleportation Card] Quantum circuit measurement: {measurement}")
        return measurement, qc


We use the following function to be able to create a card based on its Id number, this will be useful for the quantum superposition card

In [11]:
import random
def create_card_by_id(card_id):
    """Return an instance of a card based on its ID."""

    colors = ["Red", "Green", "Blue", "Yellow"]

    # Importer localement ici pour éviter les circular imports
    if card_id == 15:
        from uno.cards.Quantum_superposed_card import Quantum_superposed_card
        return Quantum_superposed_card(random.choice(colors), 8)

    card_map = {
        0: Card(random.choice(colors), 0),
        1: Card(random.choice(colors), 1),
        2: Card(random.choice(colors), 2),
        3: Card(random.choice(colors), 3),
        4: Card(random.choice(colors), 4),
        5: Card(random.choice(colors), 5),
        6: Card(random.choice(colors), 6),
        7: Card(random.choice(colors), 7),
        8: Card(random.choice(colors), 8),
        9: Card(random.choice(colors), 9),
        10: quantum_balance_card(random.choice(colors)),
        11: quantum_card(random.choice(colors), random.randint(0, 9)),
        12: Quantum_color_card(colors, random.choice(colors)),
        13: Quantum_draw_card(random.choice(colors), 8),
        14: Quantum_shuffle_card(random.choice(colors)),
        16: Quantum_swap_card(random.choice(colors)),
        17: Quantum_grover_card(random.choice(colors)),
        18: teleportation_card(random.choice(colors)),
    }

    return card_map.get(card_id, None)


### Quantum superposed card
The quantum superposed card is a card whose you don't know the effect before playing it
It uses a superposed circuit that will generate a number.
The number obtained modulo the number of different cards determines which card will be played as the result of this operation will be the cardId of the cart you will play

In [12]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

class Quantum_superposed_card(Card):
    """A card that creates a superposition over all possible card IDs and adds one to the next player."""
    def __init__(self, color, max_card_id=18):
        super().__init__(color, "Quantum Superposed Card")
        self.max_card_id = max_card_id
        self.cardId = 15

    def play(self, game):
        """Generate a card ID via quantum effect and add the corresponding card to the next player's hand."""
        card_id = self.activate_quantum_effect()
        print(f"[Quantum Superposed Card] Generated card ID: {card_id}")

        # Create the corresponding card
        card = create_card_by_id(card_id)

        if card:
            print(f"[Quantum Superposed Card] Added card with ID {card_id} to next player.")
            game.get_next_player().Hand.append(card)
        else:
            print(f"[Quantum Superposed Card] Invalid card ID generated: {card_id}")

    def activate_quantum_effect(self):
        """Quantum generation of an integer between 0 and max_card_id inclusive."""
        n_qubits = 5  # 2^5 = 32 > max_card_id
        qc = QuantumCircuit(n_qubits, n_qubits)

        # Put all qubits into superposition
        for q in range(n_qubits):
            qc.h(q)

        qc.measure(range(n_qubits), range(n_qubits))

        # Show the original circuit
        print("[Quantum Superposed Card] Original circuit:")
        print(qc.draw(output='text'))

        # Transpile for the AerSimulator
        sim = AerSimulator()
        transpiled_qc = transpile(qc, sim)

        # Show the transpiled circuit
        print("[Quantum Superposed Card] Transpiled circuit:")
        print(transpiled_qc.draw(output='text'))

        # Run it
        result = sim.run(transpiled_qc, shots=1).result()
        counts = result.get_counts()
        print(f"[Quantum Superposed Card] Quantum circuit result counts: {counts}")

        bitstring = next(iter(counts))
        number = int(bitstring, 2) % (self.max_card_id + 1)
        print(f"[Quantum Superposed Card] Measured bitstring: {bitstring} -> cardId {number}")
        print(f"[Quantum Superposed Card] Returning card ID: {number}")

        return number


### The deck
This class is used to define the deck we will use throughout the game

In [13]:
class Deck:

    def __init__(self):
        """Initialize the deck with an empty pile and discard list."""
        self.CardInPile = []
        self.CardDiscarted = []
        self.AllTheCardType = []
    def PlayCard(self, card):
        # move a played card to discard pile
        self.CardDiscarted.append(card)
    def AddCard(self, card):
        """Add a card to the deck."""
        self.CardInPile.append(card)
    def DrawCard(self):
        # draw top card from pile
        return self.CardInPile.pop(0) if self.CardInPile else None
    def GetPileSize(self):
        """Return the number of cards in the pile."""
        return len(self.CardInPile)

### The players 
This class is used to define the properties of the players who will play throughout the game

In [14]:
class Player:
    def __init__(self, name):
        """Initialize the player with a name, an empty Hand, and an empty list of played cards."""
        self.Name = name
        self.Hand = []
        self.CardPlayed = []
        self.TurnNumber = None  # initialize turn number

    def AddCard(self, card):
        """Add a card to the player's Hand."""
        self.Hand.append(card)

    def PlayCard(self, card_index):
        """Play a card from the player's Hand by index."""
        if 0 <= card_index < len(self.Hand):
            card = self.Hand.pop(card_index)
            self.CardPlayed.append(card)
            return card
        else:
            raise IndexError("Invalid card index.")

    def DrawCard(self, card):
        """Add a card to the player's Hand."""
        self.Hand.append(card)

    def GetHand(self):
        """Return the player's Hand."""
        return self.Hand

    def GetName(self):
        """Return the player's name."""
        return self.Name

    def GetCardPlayed(self):
        """Return the cards played by the player."""
        return self.CardPlayed
    
    def GetCard(sef, cardIndex):
        return self.Hand[cardIndex] if 0 <= cardIndex < len(self.Hand) else None
    
    def is_bot(self):
        """Check if the player is a bot."""
        return self.Name=="Bot" or self.Name=="BotCode"

### The Game 
This class is used to define all the functions which will be used during the game.
It contains a quantum function to determine the winning condition using quantum to find the position of the winning player.
It also contains a quantum shuffle function that uses the same principle as the quantum shuffle card but for the whole deck, it is mainly used at the start of a game

In [15]:
import random
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

class Game:
    def __init__(self):
        """Initialize the game with an empty player list, turn list, and card in play."""
        self.deck = Deck()
        self.discard_pile = []
        self.players = []
        self.current_player_idx = 0
    def add_player(self, name):
        """Add a player by name."""
        self.players.append(Player(name))

    def build_deck(self):
        """Populate and shuffle the deck."""
        colors = ["Red", "Green", "Blue", "Yellow"]
        values = [str(n) for n in range(0, 10)]
        # Add number cards
        for color in colors:
            self.deck.CardInPile.append(Card(color, "0"))
            for v in values[1:]:
                self.deck.CardInPile.append(Card(color, v))
                self.deck.CardInPile.append(Card(color, v))
        self.deck.CardInPile.append(Quantum_color_card(["Red", "Blue", "Green", "Yellow"],"Quantum"))
        self.deck.CardInPile.append(Quantum_color_card(["Red", "Blue", "Green", "Yellow"],"Quantum"))
        self.deck.CardInPile.append(Quantum_color_card(["Red", "Blue", "Green", "Yellow"],"Quantum"))
        self.deck.CardInPile.append(Quantum_color_card(["Red", "Blue", "Green", "Yellow"],"Quantum"))
        self.deck.CardInPile.append(Quantum_draw_card("Red", 8))
        self.deck.CardInPile.append(Quantum_draw_card("Blue", 8))
        self.deck.CardInPile.append(Quantum_draw_card("Green", 8))
        self.deck.CardInPile.append(Quantum_draw_card("Yellow", 8))
        self.deck.CardInPile.append(Quantum_superposed_card("Red", 8))
        self.deck.CardInPile.append(Quantum_superposed_card("Blue", 8))
        self.deck.CardInPile.append(Quantum_superposed_card("Green", 8))
        self.deck.CardInPile.append(Quantum_superposed_card("Yellow", 8))
        self.deck.CardInPile.append(Quantum_swap_card("Red"))
        self.deck.CardInPile.append(Quantum_swap_card("Blue"))
        self.deck.CardInPile.append(Quantum_swap_card("Green"))
        self.deck.CardInPile.append(Quantum_swap_card("Yellow"))
        self.deck.CardInPile.append(Quantum_shuffle_card("Red"))
        self.deck.CardInPile.append(Quantum_shuffle_card("Blue"))
        self.deck.CardInPile.append(Quantum_shuffle_card("Green"))
        self.deck.CardInPile.append(Quantum_shuffle_card("Yellow"))
        # self.deck.CardInPile.append(quantum_balance_card("Red"))
        # self.deck.CardInPile.append(quantum_balance_card("Blue"))
        # self.deck.CardInPile.append(quantum_balance_card("Green"))
        # self.deck.CardInPile.append(quantum_balance_card("Yellow"))
        self.deck.CardInPile.append(quantum_card("Red", 8))
        self.deck.CardInPile.append(quantum_card("Blue", 8))
        self.deck.CardInPile.append(quantum_card("Green", 8))
        self.deck.CardInPile.append(quantum_card("Yellow", 8))
        self.deck.CardInPile.append(Quantum_grover_card("Red"))
        self.deck.CardInPile.append(Quantum_grover_card("Blue"))
        self.deck.CardInPile.append(Quantum_grover_card("Green"))
        self.deck.CardInPile.append(Quantum_grover_card("Yellow"))
        self.deck.CardInPile.append(teleportation_card("Red"))
        self.deck.CardInPile.append(teleportation_card("Blue"))
        self.deck.CardInPile.append(teleportation_card("Green"))
        self.deck.CardInPile.append(teleportation_card("Yellow"))

        #self.QuantumShuffleDeck()
        random.shuffle(self.deck.CardInPile)

    def deal(self):
        """Deal 7 cards to each player"""
        for _ in range(7):
            for player in self.players:
                if self.deck.CardInPile:
                    player.AddCard(self.deck.CardInPile.pop(0))

    def start(self):
        """Initial setup: build deck, deal, and flip first card."""
        self.build_deck()
        self.deal()
        # flip first card
        if self.deck.CardInPile:
            first = self.deck.CardInPile.pop(0)
            self.discard_pile.append(first)

    def get_current_player(self):
        return self.players[self.current_player_idx]
    
    def get_next_player(self):
        return self.players[(self.current_player_idx + 1) % len(self.players)]

    def get_top_card(self):
        return self.discard_pile[-1] if self.discard_pile else None

    def draw_card(self, player_idx):
        """Controller handles drawing a card."""
        if self.deck.CardInPile:
            card = self.deck.CardInPile.pop(0)
            self.players[player_idx].AddCard(card)
            return card
        return None

    def play_card(self, player_idx, card_idx):
        """Controller handles playing a card."""
        played = self.players[player_idx].PlayCard(card_idx)
        played.play(self)
        self.discard_pile.append(played)
        return played

    def next_turn(self):
        self.current_player_idx = (self.current_player_idx + 1) % len(self.players)

    def quantum_detect_winner_index(self):
        """
        Simule un circuit quantique où l'index du joueur avec main vide est encodé en binaire.
        Suppose qu'un seul joueur a une main vide.
        """
        num_players = len(self.players)
        num_qubits = (num_players - 1).bit_length()

        # Cherche index du joueur gagnant
        winner_index = None
        for i, player in enumerate(self.players):
            if not player.GetHand():
                winner_index = i
                break

        if winner_index is None:
            return None  # Aucun gagnant

        # Encode l’index dans un registre quantique
        qc = QuantumCircuit(num_qubits, num_qubits)
        bin_index = format(winner_index, f'0{num_qubits}b')

        for i, bit in enumerate(reversed(bin_index)):
            if bit == '1':
                qc.x(i)

        qc.measure(range(num_qubits), range(num_qubits))

        sim = AerSimulator()
        qc = transpile(qc, sim)
        result = sim.run(qc, shots=1).result()
        counts = result.get_counts()

        measured = list(counts.keys())[0]
        return int(measured, 2)  # Reverse for Qiskit's LSB convention

    def has_winner(self):
        """Return winning player or None."""
        idx = self.quantum_detect_winner_index()
        if idx is not None:
            return self.players[idx]
        return None
    
    def QuantumShuffleDeck(self):
        """Shuffle the deck using quantum principles."""
        shuffled_deck = []
        """Generate a quantum random number between 0 and max_cards (inclusive)."""
        n_qubits = 3  #
        qc = QuantumCircuit(n_qubits, n_qubits)

        # Put all qubits into superposition
        for q in range(n_qubits):
            qc.h(q)

        qc.measure(range(n_qubits), range(n_qubits))
        sim = AerSimulator()    
        qc = transpile(qc, sim)
        job = sim.run(qc)
        result = job.result()
        counts = result.get_counts()

        bitstring = list(counts.keys())[0]
        number = int(bitstring, 2)
        
        while self.deck:
            shuffled_deck.append(self.deck.CardInPile.pop(number% len(self.deck.CardInPile)))
        self.deck = shuffled_deck

    def reverse_turn_order(self):
        """Reverse the turn order of players."""
        self.players.reverse()
        self.current_player_idx = len(self.players) - 1 - self.current_player_idx

    def get_card_by_idx(self, card_idx):
        player = self.get_current_player()
        if 0 <= card_idx < len(player.Hand):
            return player.Hand[card_idx]
        return None
        

# Gameplay
The following cells represent the typical unfolding of a Quantum You Know game with the creation of the game itself, the addition of players and the use of different cards

In [16]:
Game=Game()
Game.add_player("Player 1")
Game.add_player("Player 2")
Game.start()

Created quantum circuit: circuit-162
Created quantum circuit: circuit-163
Created quantum circuit: circuit-164
Created quantum circuit: circuit-165


In [23]:
CardEffect=quantum_balance_card(Game).play(Game)

Card weight for Yellow 4: 5
Card weight for Blue Quantum Grover: 5
Card weight for Yellow 5: 5
Card weight for Red 7: 5
Card weight for Yellow Quantum Hand Swap: 5
Card weight for Blue 8: 5
Card weight for Blue 2: 5
Card weight for Yellow 3: 5
Card weight for Blue 9: 5
Card weight for Yellow 1: 5
Card weight for Green 7: 5
Card weight for Red 8: 5
Card weight for Red Quantum Grover: 5
Card weight for Yellow 2: 5

Initial QAOA ansatz circuit:
      »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
q_11: »
      »
q_12: »
      »
q_13: »
      »
«      ┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
« q_0: ┤0                                                                          

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/optimize/_cobyla_py.py", line 281, in calcfc
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
    f = sf.fun(x)
        ^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py", line 278, in fun
    self._update_fun()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py", line 262, in _update_fun
    self._update_fun_impl()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py", line 163, in update_fun
    self.f = fun_wrapped(self.x)
             ^^^^^^^^

: 

In [17]:
CardEffect=quantum_card("Red", 8).play(Game)

Created quantum circuit: circuit-166
Created quantum circuit: circuit-167

Original effect circuit:
        ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 

Transpiled effect circuit:
        ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
Quantum effect result counts: {'1': 1}
Counts from quantum effect: {'1': 1}
Effect: Reversing turn order.


In [18]:
CardEffect=Quantum_color_card(["Red", "Blue", "Green", "Yellow"],"Quantum").play(Game)

Current color set to: Red
Quantum walk steps: 4
Initial position index: 0

Quantum walk circuit:
                    ┌───┐               ┌───┐          ┌───┐               »
q_0: ───────────────┤ X ├──■─────────■──┤ X ├──────────┤ X ├──■─────────■──»
                    └─┬─┘┌─┴─┐     ┌─┴─┐└─┬─┘          └─┬─┘┌─┴─┐     ┌─┴─┐»
q_1: ─────────────────┼──┤ X ├─────┤ X ├──┼──────────────┼──┤ X ├─────┤ X ├»
     ┌───┐┌───┐┌───┐  │  └─┬─┘┌───┐└─┬─┘  │  ┌───┐┌───┐  │  └─┬─┘┌───┐└─┬─┘»
q_2: ┤ H ├┤ H ├┤ X ├──■────■──┤ X ├──■────■──┤ H ├┤ X ├──■────■──┤ X ├──■──»
     └───┘└───┘└───┘          └───┘          └───┘└───┘          └───┘     »
c: 2/══════════════════════════════════════════════════════════════════════»
                                                                           »
«     ┌───┐          ┌───┐               ┌───┐          ┌───┐               »
«q_0: ┤ X ├──────────┤ X ├──■─────────■──┤ X ├──────────┤ X ├──■─────────■──»
«     └─┬─┘          └─┬─┘┌─┴─┐     ┌─┴─┐└─┬─┘        

In [19]:
CardEffect=Quantum_draw_card("Red", 8).play(Game)


Original random-number circuit:
     ┌───┐┌─┐      
q_0: ┤ H ├┤M├──────
     ├───┤└╥┘┌─┐   
q_1: ┤ H ├─╫─┤M├───
     ├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├─╫──╫─┤M├
     └───┘ ║  ║ └╥┘
c: 3/══════╩══╩══╩═
           0  1  2 

Transpiled random-number circuit:
     ┌───┐┌─┐      
q_0: ┤ H ├┤M├──────
     ├───┤└╥┘┌─┐   
q_1: ┤ H ├─╫─┤M├───
     ├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├─╫──╫─┤M├
     └───┘ ║  ║ └╥┘
c: 3/══════╩══╩══╩═
           0  1  2 
Quantum effect result counts: {'000': 1}
Measured bitstring: 000 -> interpreted number: 0
Returning valid number: 0
Number of cards to draw: 0


In [20]:
CardEffect=Quantum_grover_card("Red").play(Game)


🔍 Grover scan for: Red 0

Quantum circuit used for Grover search:
     ┌───┐┌───┐┌───┐          ┌───┐┌───┐┌───┐┌───┐               ┌───┐┌───┐»
q_0: ┤ H ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├»
     ├───┤├───┤├───┤       │  ├───┤├───┤├───┤├───┤            │  ├───┤├───┤»
q_1: ┤ H ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├»
     ├───┤├───┤├───┤       │  ├───┤├───┤├───┤├───┤            │  ├───┤├───┤»
q_2: ┤ H ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├»
     ├───┤├───┤├───┤       │  ├───┤├───┤├───┤├───┤            │  ├───┤├───┤»
q_3: ┤ H ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├»
     ├───┤├───┤├───┤       │  ├───┤├───┤├───┤├───┤            │  ├───┤├───┤»
q_4: ┤ H ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├»
     ├───┤├───┤├───┤       │  ├───┤├───┤├───┤├───┤            │  ├───┤├───┤»
q_5: ┤ H ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├»
     ├───

In [21]:
CardEffect=Quantum_shuffle_card("Red").play(Game)

Collected full deck: ['Blue 6', 'Green 6', 'Red 3', 'Quantum Color', 'Blue 5', 'Green Quantum Hand Swap', 'Red Quantum True Shuffle', 'Quantum Color', 'Blue Quantum Draw up to 8', 'Green Quantum Superposed Card', 'Blue Quantum True Shuffle', 'Red 0', 'Yellow 3', 'Yellow Quantum True Shuffle']
Quantum random index counts: {'0110': 1}
Returning quantum random index: 6
Quantum circuit for random index generation:
     ┌───┐┌─┐         
q_0: ┤ H ├┤M├─────────
     ├───┤└╥┘┌─┐      
q_1: ┤ H ├─╫─┤M├──────
     ├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─╫──╫─┤M├───
     ├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─╫──╫──╫─┤M├
     └───┘ ║  ║  ║ └╥┘
c: 4/══════╩══╩══╩══╩═
           0  1  2  3 
Selected index: 6 from full deck of size 14
Assigned card Red Quantum True Shuffle to player 0
Quantum random index counts: {'1101': 1}
Returning quantum random index: 13
Quantum circuit for random index generation:
     ┌───┐┌─┐         
q_0: ┤ H ├┤M├─────────
     ├───┤└╥┘┌─┐      
q_1: ┤ H ├─╫─┤M├──────
     ├───┤ ║ └╥┘┌─┐   
q_

In [22]:
CardEffect=Quantum_superposed_card("Red", 8).play(Game)

[Quantum Superposed Card] Original circuit:
     ┌───┐┌─┐            
q_0: ┤ H ├┤M├────────────
     ├───┤└╥┘┌─┐         
q_1: ┤ H ├─╫─┤M├─────────
     ├───┤ ║ └╥┘┌─┐      
q_2: ┤ H ├─╫──╫─┤M├──────
     ├───┤ ║  ║ └╥┘┌─┐   
q_3: ┤ H ├─╫──╫──╫─┤M├───
     ├───┤ ║  ║  ║ └╥┘┌─┐
q_4: ┤ H ├─╫──╫──╫──╫─┤M├
     └───┘ ║  ║  ║  ║ └╥┘
c: 5/══════╩══╩══╩══╩══╩═
           0  1  2  3  4 
[Quantum Superposed Card] Transpiled circuit:
     ┌───┐┌─┐            
q_0: ┤ H ├┤M├────────────
     ├───┤└╥┘┌─┐         
q_1: ┤ H ├─╫─┤M├─────────
     ├───┤ ║ └╥┘┌─┐      
q_2: ┤ H ├─╫──╫─┤M├──────
     ├───┤ ║  ║ └╥┘┌─┐   
q_3: ┤ H ├─╫──╫──╫─┤M├───
     ├───┤ ║  ║  ║ └╥┘┌─┐
q_4: ┤ H ├─╫──╫──╫──╫─┤M├
     └───┘ ║  ║  ║  ║ └╥┘
c: 5/══════╩══╩══╩══╩══╩═
           0  1  2  3  4 
[Quantum Superposed Card] Quantum circuit result counts: {'00110': 1}
[Quantum Superposed Card] Measured bitstring: 00110 -> cardId 6
[Quantum Superposed Card] Returning card ID: 6
[Quantum Superposed Card] Generated card ID: 6
Create

In [23]:
CardEffect=Quantum_swap_card("Red").play(Game)


Quantum CSWAP circuit for hand swap:
      ┌───┐                                                         ┌─┐      
 q_0: ┤ H ├─■──■────────■────────■────────■────────■────────■───────┤M├──────
      └───┘ │  │ ┌─┐    │        │        │        │        │       └╥┘      
 q_1: ──────X──┼─┤M├────┼────────┼────────┼────────┼────────┼────────╫───────
            │  │ └╥┘    │ ┌─┐    │        │        │        │        ║       
 q_2: ──────┼──X──╫─────┼─┤M├────┼────────┼────────┼────────┼────────╫───────
            │  │  ║     │ └╥┘    │ ┌─┐    │        │        │        ║       
 q_3: ──────┼──┼──╫─────X──╫─────┼─┤M├────┼────────┼────────┼────────╫───────
            │  │  ║     │  ║     │ └╥┘    │ ┌─┐    │        │        ║       
 q_4: ──────┼──┼──╫─────┼──╫─────X──╫─────┼─┤M├────┼────────┼────────╫───────
            │  │  ║     │  ║     │  ║     │ └╥┘    │ ┌─┐    │        ║       
 q_5: ──────┼──┼──╫─────┼──╫─────┼──╫─────X──╫─────┼─┤M├────┼────────╫───────
            │  │  ║     │ 

In [24]:
CardEffect=teleportation_card("Red").play(Game)

[Teleportation Card] Quantum circuit measurement: 00

Quantum teleportation circuit:
     ┌───┐          ┌───┐┌─┐
q_0: ┤ H ├───────■──┤ H ├┤M├
     ├───┤     ┌─┴─┐└┬─┬┘└╥┘
q_1: ┤ H ├──■──┤ X ├─┤M├──╫─
     └───┘┌─┴─┐└───┘ └╥┘  ║ 
q_2: ─────┤ X ├───────╫───╫─
          └───┘       ║   ║ 
c: 2/═════════════════╩═══╩═
                      1   0 
[Teleportation Card] Measurement result: 00
[Teleportation Card] Number of '1's in measurement: 0
[Teleportation Card] No swaps to perform. Returning.


In [25]:
Game.quantum_detect_winner_index()

# The  Bot
Below you can see the code of the bot that we use in our game.
This bot uses QAOA to determine the next best action to play

In [48]:
from docplex.mp.model import Model
from qiskit_optimization.translators import from_docplex_mp, to_ising
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import QAOAAnsatz
from qiskit_ibm_runtime import EstimatorV2 as Estimator, SamplerV2 as Sampler
from qiskit_aer import AerSimulator
from qiskit import transpile
from scipy.optimize import minimize
import numpy as np
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

class BotCode(Player):
    def __init__(self, name="Bot"):
        super().__init__(name)
        self.is_bot = True

    def card_weight(self, card):
        if card.value == 'Draw Two':
            return 20
        elif card.value in ['Skip', 'Reverse']:
            return 10
        elif isinstance(card.value, int):
            return 1
        else:
            return 5

    def is_playable(self, card, top_card):
        return (
            card.color == top_card.color
            or card.value == top_card.value
            or top_card.color == "Quantum"
            or card.color == "Quantum"
        )

    def generate_qubo(self, Hand, top_card):
        mdl = Model("BotDecision")
        m = len(Hand)
        x = [mdl.binary_var(name=f"x_{i}") for i in range(m)]

        print(f"\n[QUBO] Hand playable mask: {[str(card) if self.is_playable(card, top_card) else 0 for card in Hand]}")
        weights = [self.card_weight(card) if self.is_playable(card, top_card) else 0 for card in Hand]
        print(f"[QUBO] Weights: {weights}")

        obj = mdl.sum(-weights[i] * x[i] for i in range(m))
        constraint = mdl.sum(x) - 1
        alpha = 100
        mdl.minimize(obj + alpha * constraint * constraint)

        qp = from_docplex_mp(mdl)
        qubo = QuadraticProgramToQubo().convert(qp)
        print(f"[QUBO] QuadraticProgram: {qp}")
        print(f"[QUBO] Converted to QUBO:\n  linear: {qubo.objective.linear.to_array()}\n  quadratic:\n{qubo.objective.quadratic.to_array()}\n  offset: {qubo.objective.constant}")
        return qubo, weights

    def cost_function(self, params, estimator, circuit, hamiltonian, pass_manager):
        # transpile & run through pass manager so we see final layout
        transpiled = pass_manager.run(transpile(circuit))
        print(f"\n[COST] Transpiled circuit for cost eval:\n{transpiled.draw(output='text')}")
        # apply layout to Hamiltonian
        isa_observables = hamiltonian.apply_layout(transpiled.layout)
        job = estimator.run([(transpiled, isa_observables, params)])
        cost = job.result()[0].data.evs
        print(f"[COST] Params: {params}\n[COST] Energy: {cost}")
        return cost

    def decide_action(self, game):
        top_card = game.get_top_card()
        Hand = self.GetHand()

        if not Hand:
            print("[ACTION] Empty hand → drawing")
            return ('draw', None)

        qubo, weights = self.generate_qubo(Hand, top_card)
        hamiltonian, _ = to_ising(qubo)

        backend = AerSimulator()
        estimator = Estimator(mode=backend)
        sampler = Sampler(mode=backend)
        pass_manager = generate_preset_pass_manager(backend=backend, optimization_level=3)

        # build QAOA ansatz
        circuit_qaoa = QAOAAnsatz(hamiltonian, reps=10)
        print(f"\n[ANSATZ] QAOAAnsatz (depth=10):\n{circuit_qaoa.draw(output='text')}")

        # also show transpiled ansatz
        transpiled_ansatz = transpile(circuit_qaoa, backend=backend)
        print(f"[ANSATZ] Transpiled ansatz:\n{transpiled_ansatz.draw(output='text')}")

        # initial params
        p = circuit_qaoa.num_parameters // 2
        gamma_init = np.linspace(0.1, 1.5, p)
        beta_init = np.linspace(0.1, 1.5, p)
        params_init = np.concatenate([gamma_init, beta_init])
        print(f"[OPT] Initial gamma: {gamma_init}")
        print(f"[OPT] Initial beta:  {beta_init}")

        # optimize
        res_opt = minimize(
            self.cost_function,
            params_init,
            args=(estimator, circuit_qaoa, hamiltonian, pass_manager),
            method="COBYLA"
        )
        params_opt = res_opt.x
        print(f"[OPT] Optimization result:\n  success: {res_opt.success}\n  fun: {res_opt.fun}\n  x: {params_opt}")

        # build measurement circuit
        circuit_meas = circuit_qaoa.decompose(reps=2).copy()
        circuit_meas.measure_all()
        print(f"\n[MEASURE] Measurement circuit:\n{circuit_meas.draw(output='text')}")

        # sample
        counts = sampler.run([(circuit_meas, params_opt)]).result()[0].data.meas.get_counts()
        print(f"[SAMPLE] Raw counts: {counts}")

        # pick bitstring
        sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)
        if sorted_counts:
            most_likely, second = sorted_counts[0], (sorted_counts[1] if len(sorted_counts) > 1 else (None, None))
            bit0, cnt0 = most_likely
            bit1, cnt1 = second
            if bit0 == "0" * len(bit0):
                print("[SAMPLE] Most likely all-zero; switching to second best")
                most_likely_bitstring = bit1
            else:
                most_likely_bitstring = bit0
            print(f"[SAMPLE] Chosen bitstring: {most_likely_bitstring} (count {cnt0})")
        else:
            most_likely_bitstring = None
            print("[SAMPLE] No counts returned")

        # decide which card to play
        if most_likely_bitstring:
            for i, bit in enumerate(reversed(most_likely_bitstring)):
                print(f"[MAP] Bit[{i}]={bit} → Card[{i}]={Hand[i]}")
                if bit == '1' and self.is_playable(Hand[i], top_card):
                    print(f"[DECIDE] Playing card index {i}: {Hand[i]}")
                    return ('play', i)

        # fallback: highest-weight playable card
        playable = [i for i, c in enumerate(Hand) if self.is_playable(c, top_card)]
        if playable:
            best = max(playable, key=lambda i: self.card_weight(Hand[i]))
            print(f"[FALLBACK] Playing highest-weight card index {best}: {Hand[best]}")
            return ('play', best)

        print("[DECIDE] No playable → drawing")
        return ('draw', None)

    def take_turn(self, game):
        idx = game.players.index(self)
        action, card_idx = self.decide_action(game)
        if action == 'play':
            played = game.play_card(idx, card_idx)
            print(f"[TURN] Played {played}")
            return ('play', played)
        drawn = game.draw_card(idx)
        print(f"[TURN] Drew {drawn.__str__() if drawn else 'None'}")
        return ('draw', drawn)

    def is_bot(self):
        """Check if the player is a bot."""
        return True


In [49]:
# Mock game class to simulate basic UNO logic
class MockGame:
    def __init__(self, top_card, players):
        self.top_card = top_card
        self.players = players

    def get_top_card(self):
        return self.top_card

    def play_card(self, player_idx, card_idx):
        played = self.players[player_idx].GetHand().pop(card_idx)
        self.top_card = played  # Update top card
        return played

    def draw_card(self, player_idx):
        drawn = Card("Red", 3)
        self.players[player_idx].GetHand().append(drawn)
        return drawn
    
# Create a bot and a test game
bot = BotCode()
bot.hand = [
    Card("Red", 3),
    Card("Blue", "Skip"),
    Card("Yellow", "Draw Two"),
    Card("Red", 5),
    Card("Green", 9)
]

top_card = Card("Red", 7)
game = MockGame(top_card, [bot])

# Run the bot's turn
for i in range(2):  # Simulate 3 turns
    print(f"\n--- Turn {i+1} ---")
    bot.take_turn(game)



--- Turn 1 ---
[ACTION] Empty hand → drawing
[TURN] Drew Red 3

--- Turn 2 ---

[QUBO] Hand playable mask: ['Red 3']
[QUBO] Weights: [1]
[QUBO] QuadraticProgram: minimize 100*x_0^2 - 201*x_0 + 100 (1 variables, 0 constraints, 'BotDecision')
[QUBO] Converted to QUBO:
  linear: [-201.]
  quadratic:
[[100.]]
  offset: 100

[ANSATZ] QAOAAnsatz (depth=10):
   »
q: »
   »
«   ┌───────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q: ┤ QAOA(γ[0],β[0],γ[1],β[1],γ[2],β[2],γ[3],β[3],γ[4],β[4],γ[5],β[5],γ[6],β[6],γ[7],β[7],γ[8],β[8],γ[9],β[9]) ├
«   └───────────────────────────────────────────────────────────────────────────────────────────────────────────┘
[ANSATZ] Transpiled ansatz:
   ┌───┐┌────────────────┐┌──────────────┐┌────────────────┐┌──────────────┐»
q: ┤ H ├┤ Rz(101.0*γ[0]) ├┤ Rx(2.0*β[0]) ├┤ Rz(101.0*γ[1]) ├┤ Rx(2.0*β[1]) ├»
   └───┘└────────────────┘└──────────────┘└────────────────┘└──────────────┘»
«   ┌────────────────┐┌───